In [ ]:
import requests
import os
import json
import pandas as pd
import warnings
import time
from IPython.display import clear_output
warnings.filterwarnings("ignore")

# Authentication

In [ ]:
os.environ['BEARER_TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAPprQwEAAAAATVpkccH9wayqAHIchStFOxPDcVQ%3DxCDU7ucO06tFk0iipxUTURwSYy5KGmruzXFEajWjHwyqUVgDry'
os.environ['CONSUMER_KEY'] = 'Se8VbnexMEOSbem7C4ddQfiKb'
os.environ['CONSUMER_SECRET'] = 'DFwM5bOREWODkobpxYrOveXALgE0xb28sNmJVfiljqoHw0BtPv'

In [ ]:
def auth():
    return os.environ.get("BEARER_TOKEN")

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    try:
      response = requests.request("GET", url, headers=headers, params=params)
      print("status code: ", response.status_code)
      if response.status_code != 200:
          raise Exception(
              "Request returned an error: {} {}".format(
                  response.status_code, response.text
              )
          )
    except:
        print("Timeout: sleep for 15 minutes and try again")
        time.sleep(15 * 60)
        response = requests.request("GET", url, headers=headers, params=params)
    return response.json()

# Tweets API

In [ ]:
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/sample-realtime/overview
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/filter-realtime/api-reference/post-statuses-filter
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent

# Tweets Data

In [ ]:
def tweets(keyword, token_flag = False, max_id = ""):
    if (token_flag):
        bearer_token = auth()
        url = "https://api.twitter.com/1.1/search/tweets.json"
        params = {"q":keyword, "count":"100", "max_id":max_id, "include_entities":1}
        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(url, headers, params)
        return(json_response)
    else:
        bearer_token = auth()
        url = "https://api.twitter.com/1.1/search/tweets.json"
        params = {"q":keyword, "count":"100"}
        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(url, headers, params)
        return(json_response)

In [ ]:
keywords = ["Telkomsel", "myXLCare", "IndosatCare"]

In [ ]:
df_tweets = pd.DataFrame()

for key in keywords:
    data = tweets(key)

    # mengatasi error apabila user private
    if (('errors' in data) or (data['search_metadata']['count'] == 0)):
        continue

    # mengubah data json menjadi dataframe
    df = pd.DataFrame(data['statuses']).pop("text")

    # mengambil next tweet berdasar next token
    while ((len(df)<1000) & (data['search_metadata']['count']==100)):
        next_results = data['search_metadata']['next_results']
        max_id = next_results.split('=', 1)[1].split('&', 1)[0]

        data = tweets(key, True, max_id)

        # apabila ternyata tweet telah habis dan data masih kurang dari 1000 (pembatalan request)
        if (data['search_metadata']['count']==0):
            break

        df_temp = pd.DataFrame(data['statuses']).pop("text")

        df = pd.concat([df, df_temp])
        clear_output(wait=True)
        print('Total data - {}: '.format(key), len(df), '/ 1000')

    # penggabungan df
    df_tweets=pd.concat([df_tweets, df])  
    clear_output(wait=True)

    # mengubah dataframe jadi csv
    df_tweets.to_csv(r'{}.csv'.format(key), index = False)

    # reset dataframe
    df_tweets = df_tweets.iloc[0:0]

Total data - IndosatCare:  1000 / 1000
